In [1]:
import math
from iapws import IAPWS97

In [ ]:
def calc_thermo(node):
    '''Calculate Steam T_sat, Rho and Mu from steam table'''
    if pipe_dict[pipeline].is_stl():
        node_dict[node_in].steam_p()
    else:
        node_dict[node_in].brine_t()
        
    print(pipeline, pipe_dict[pipeline].is_stl(), p_f)
    
    '''Calculate area and velocity of inlet node; Only area for outlet node'''
    node_dict[node_in].calc_area()
    node_dict[node_in].calc_velocity(node_dict[node_in])
    node_dict[node_out].calc_area()
    #node_dict[node_out].calc_velocity(node_dict[node_in])
    #print('{} = {:.4f}'.format(node_dict[node_in].ID, node_dict[node_in].v))
    
    '''Calculate Reynolds Number at the inlet node'''
    node_dict[node_in].nRE()
    #print('{:.0f}'.format(node_dict[node_in].reynolds))

In [ ]:
def calc_dp():
    '''Calculate Relative Roughness of the pipeline'''
    pipe_dict[pipeline].rel_roughness(node_dict[node_in].d)
    
    '''Calculate friction factor of the pipe'''
    if node_dict[node_in].reynolds > 0: 
        pipe_dict[pipeline].ff_fanning(node_dict[node_in].reynolds)
        #print('{}'.format(pipe_dict[pipeline].ff))
    else:
        pass
    
    

In [ ]:
def iter2_5(node_in, node_out, pipe, dpi):
    
    #(1) Assume pressure drop
    dpi = 0.001
    
    #(2) calc P, T, hf and hg of node_out
    node_out.p = node_in.p + dpi
    steam_out = IAPWS97(P = node_out.p, x=1)
    node_out.hg =steam_out.h #sat'd steam enthalpy
    node_out.t = steam_out.T
    steam_out = IAPWS97(P = node_out.p, x=0)
    node_out.hf = steam_out.h #sat'd liq enthalpy
    
    #(3) calc X and node_out.mf
    #calc Q
    pipe.q_loss(node_in, insul)
    #pipe.Q = pipe.Q*1e-3  #convert to kJoules
    H = node_in.h + pipe.Q*1e-3
    X = (H - node_out.hf)/(node_out.hg - node_out.hf)
    node_out.mf = node_in.mf*X
    print('calc X and node_out.mf', pipe.Q, node_in.mf, node_out.mf)
    
    #(4) recalculate v, udu, f_tot
    node_out.steam_p() #calculate steam parameters
    print('node_out', node_out.rho)
    node_out.calc_area() #calculate area
    node_out.calc_velocity(node_out) #recalculate velocity @ new mf
    node_out.nRE() #recalculate nRE @ new v 
    print('New velocity = %f MF = %f'%(node_out.v, node_out.mf))
    
    pipe.ff_fanning(node_out.reynolds)
    pipe.skin_friction(node_out.v, node_out.d)
    
    if node_in.area == node_out.area:
        pipe.f_exp = 0
        pipe.f_con = 0
    
    if node_in.area < node_out.area:
        pipe.f_con = 0
        pipe.f_expansion(node_in.area, node_out.area, node_in.v)
    
    else:
        pipe.f_exp = 0
        pipe.f_contraction(node_in.area, node_out.area, node_in.v)
    
    pipe.f_fittings(kf_list, node_in.v)
    pipe.f_total()

    pipe.f_gdz(node_in.z, node_out.z)
    pipe.f_udu(node_in.v, node_out.v)
    
    pipe.total_loss()
    
    p2 = node_in.p + (pipe.udu + pipe.gdz - pipe.f_tot) * node_out.rho * 1e-6
    dpf = p2 - node_in.p
    #dpf = (pipe.udu + pipe.gdz - pipe.f_tot) * node_in.rho * 1e-6
    
    #print(pipe.f_skin, node_out.v, node_out.mf, node_out.rho, node_out.area)
    #print(pipe.f_tot, pipe.t_loss, pipe.Q, p2)
    #print(abs(dpi), abs(dpf))
    print('Final pipe; f_tot=%f, f_skin=%f, f_fit=%f, udu=%f'%(pipe.f_tot, pipe.f_skin, pipe.f_fit, pipe.udu))
    print('Initial parameters; Node_out: P=%f, MF=%f, T=%f, v=%f, rho=%f'%\
          (node_out.p, node_out.mf, node_out.t, node_out.v, node_out.rho))
    return [p2, dpf, dpi-dpf]